In [ ]:
from typing import Literal
import numpy as np
import xarray as xr


np.set_printoptions(formatter={"float": lambda x: "{0:0.2f}".format(x)})

## Pitch: construct the Equal Temperament

In [ ]:
from typing import get_args

# Reference https://www.youtube.com/watch?v=rhQ8zF4VCnY


NoteName = Literal["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
ALL_NOTES = get_args(NoteName)


def construct_equal_temperament_frequency_table(
    reference_note_name: NoteName = "A",
    reference_note_octave: int = 3,
    reference_note_frequency: float = 440,
    octave_range_min_inclusive: int = -1,
    octave_range_max_inclusive: int = 9,
    note_names: tuple[NoteName] = ALL_NOTES,
):
    octaves = np.arange(octave_range_min_inclusive, octave_range_max_inclusive + 1)
    # 0 = C (Do) ; 11 = B (Si)
    notes = np.array(list(note_names))
    position = (
        reference_note_octave - octave_range_min_inclusive
    ) * notes.size + note_names.index(reference_note_name)
    array = xr.DataArray(
        np.zeros((octaves.size, notes.size)),
        dims=("octave", "note"),
        coords={"octave": octaves, "note": notes},
    ).stack(z=("octave", "note"))
    frequencies = reference_note_frequency * np.power(
        2, (np.arange(array.size) - position) / 12
    )
    frequency_table = array.copy(data=frequencies).unstack()
    return frequency_table


frequency_table = construct_equal_temperament_frequency_table()
assert frequency_table.sel(note="A", octave=3) == 440
frequency_table

In [ ]:
frequency_table.plot()

In [ ]:
np.log2(frequency_table).plot()

## Tempo: construct the polyrythms

In [ ]:
NOTES_COUNT = 25

# coords
integer_range = np.arange(NOTES_COUNT)
# Create notes
xr.DataArray(data=integer_range, dims="pitch")

# Period in ms
T = 1000
# Delta period in ms
ΔT = 125
# Create periods with a small dt
periods = 1000 + (ΔT // NOTES_COUNT) * integer_range
print(periods)
np.lcm.reduce(periods)

In [ ]:
periods_xda = xr.DataArray(data=periods, dims="voice")
periods_xda

In [ ]:
t = xr.DataArray(np.arange(0, 15), dims="t")
polyrythm = t * periods_xda
display(polyrythm)

polyrythm.plot.line(y="voice", marker="o", hue=None)

## TODO: Construct lines between lowest and highest notes instead of the 25 points ; use observable plot with the line plot.

In [ ]:
np.lcm(1000, 1200)

In [ ]:
np.lcm.reduce(periods)